In [5]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
        
        
train_data = pd.read_csv(r"C:\Users\Monish Kumar\Python projects\Basic Language Processing\nltk_input_data\train_data.csv")
test_data = pd.read_csv(r"C:\Users\Monish Kumar\Python projects\Basic Language Processing\nltk_input_data\test_data.csv")

train_data = train_data.drop(columns=['Unnamed: 0'])
test_data = test_data.drop(columns=['Unnamed: 0'])

In [6]:
# regularization constant...keep_prob
# layer_1 = tf.nn.relu(layer_1)...
# layer_1 = tf.nn.dropout(layer_1,keep_prob)

# sigmoid instead of relu??...bias working and why is it used?...

In [7]:
X_train = pd.DataFrame(train_data.drop(columns=['pos','neg']))
Y_train = pd.DataFrame(train_data[['pos','neg']])

X_test = pd.DataFrame(test_data.drop(columns=['pos','neg']))
Y_test = pd.DataFrame(test_data[['pos','neg']])

In [8]:
nodes_1 = 500
nodes_2 = 500
nodes_3 = 500
class_size = 2      # positive and negative
feature_size = 423  # length of lexicon
batch_size = 100

x = tf.placeholder('float',[None,feature_size])  # no of rows is not specified
y = tf.placeholder('float')                      # no of rows and columns are not specified
training_output = tf.placeholder('float',[None,class_size])
submission_output = tf.placeholder('float',[None,class_size])

In [9]:
def neural_network(input_data):
    
    hidden_layer_1 = {'weights':tf.Variable(tf.random.uniform([feature_size,nodes_1])),
                      'bias':tf.Variable(tf.random.uniform([nodes_1]))}
    hidden_layer_2 = {'weights':tf.Variable(tf.random.uniform([nodes_1,nodes_2])),
                          'bias':tf.Variable(tf.random.uniform([nodes_2]))}
    hidden_layer_3 = {'weights':tf.Variable(tf.random.uniform([nodes_2,nodes_3])),
                          'bias':tf.Variable(tf.random.uniform([nodes_3]))}
    output_layer = {'weights':tf.Variable(tf.random.uniform([nodes_3,class_size])),
                          'bias':tf.Variable(tf.random.uniform([class_size]))}
    
    # (data * weights) + bias
    l1 = tf.add(tf.linalg.matmul(input_data,hidden_layer_1['weights']),hidden_layer_1['bias'])
    l1 = tf.nn.relu(l1)  # rectified linear function...similar to sigmoid function(tf.nn.sigmoid())
    
    l2 = tf.add(tf.linalg.matmul(l1,hidden_layer_2['weights']),hidden_layer_2['bias'])
    l2 = tf.nn.relu(l2)
    
    l3 = tf.add(tf.linalg.matmul(l2,hidden_layer_3['weights']),hidden_layer_3['bias'])
    l3 = tf.nn.relu(l3)
    
    output = tf.add(tf.linalg.matmul(l3,output_layer['weights']),output_layer['bias'])
    output = tf.nn.relu(output)
    
    return output
    

In [10]:
def train_neural_network():
    
    # Defining the graph for model...
    prediction = neural_network(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y,logits = prediction))
    optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
    
    # Defining graph for accuracy calculation...
    correct_prediction = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction ,'float'))
        
    epochs_num = 10
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(epochs_num):
            epoch_loss = 0
            for n in range(int(X_train.shape[0]/batch_size)):
                epoch_x = X_train.loc[(n*batch_size):((n+1)*batch_size)]
                epoch_y = Y_train.loc[(n*batch_size):((n+1)*batch_size)]
                _, c = sess.run([optimizer, cost], feed_dict = {x:epoch_x, y:epoch_y})
                epoch_loss += c
            print('Epoch',epoch+1,'completed out of',epochs_num,'loss: ',epoch_loss)
            
        
        print('\nAccuracy on Training data : ',accuracy.eval(feed_dict = {x:X_train, y:Y_train})) 
        print('Accuracy on Testing data : ',accuracy.eval(feed_dict = {x:X_test, y:Y_test})) 
        
        #training_output = sess.run(tf.argmax(prediction,1),feed_dict = {x:X_train})   # using sess.run() ...but both produces the same result
        #training_output = tf.argmax(prediction,1).eval(feed_dict = {x:X_train})        # using .eval() ...both can only work under with tf.Session() as sess:
        #submission_output = sess.run(tf.argmax(prediction,1),feed_dict = {x:X_test})
        
        print('\ndone...')
        

In [11]:
# train and display loss and accuracy
train_neural_network()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch 1 completed out of 10 loss:  2742644.6991271973
Epoch 2 completed out of 10 loss:  1910092.4030456543
Epoch 3 completed out of 10 loss:  1196826.8077392578
Epoch 4 completed out of 10 loss:  872371.3898773193
Epoch 5 completed out of 10 loss:  897752.4040298462
Epoch 6 completed out of 10 loss:  620831.3835449219
Epoch 7 completed out of 10 loss:  703831.4662017822
Epoch 8 completed out of 10 loss:  461216.32232666016
Epoch 9 completed out of 10 loss:  368526.47956085205
Epoch 10 completed out of 10 loss:  337104.1943664551

Accuracy on Training data :  0.50257915
Accuracy on Testing data :  0.48968107

done...


In [24]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter

lexicon_data = pd.read_csv(r"C:\Users\Monish Kumar\Python projects\Basic Language Processing\nltk_input_data\lexicon_data.csv")
lexicon = list(lexicon_data['0'])

t = 'you are a not a  fuck fuck fuck fuck fuck boy'
t = t.lower()
words = word_tokenize(t)
#print('Before lemmatization -',words)

lemmed_words = []
for i in words:
    lemmed_words.append(WordNetLemmatizer().lemmatize(i)) 
#print('After lemmatization -',lemmed_words)

#print(Counter(lemmed_words))

input_demo = np.zeros(len(lexicon))
for w in lemmed_words:
    if w in lexicon:
        pos = lexicon.index(w)
        input_demo[pos] += 1
#print('Sample Input data -',input_demo)

In [25]:
pred = neural_network(tf.cast(pd.DataFrame(input_demo.reshape(1,-1)),'float'))
out = tf.argmax(pred,1)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    p,o = sess.run([pred,out])
    
if o == 0:
    print('Positive Statement')
else:
    print('Negative Statement')

Negative Statement


In [26]:
print ('Probability of being a positive Statement -',round(p[0,0]/(p[0,0] + p[0,1]),2),'%')
print ('Probability of being a negative Statement -',round(p[0,1]/(p[0,0] + p[0,1]),2),'%')

Probability of being a positive Statement - 0.49 %
Probability of being a negative Statement - 0.51 %


Tutorial using 1.6 million samples of data - https://www.youtube.com/watch?v=JeamFbHhmDo&list=PLQVvvaa0QuDfKTOs3Keq_kaG2P55YRn5v&index=51  
If you see the last 3 min of the above video you will notice that the even though we used bigger amount of dataset we only reached an accuracy of ~74%
### Conclusion
Natural language processing doesn't perform well on traditional feed-forward back-prob neural networks. It may perform well on Recurrent Neural Networks (or) LSTMs (or) other algos...